<a href="https://colab.research.google.com/github/Jasper-Hewitt/homework/blob/main/Jiabo_roberta_fakenews_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 Load dependences, data set

In [ ]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 32.6 MB/s 
     |████████████████████████████████| 132 kB 75.0 MB/s 
     |████████████████████████████████| 212 kB 73.0 MB/s 
     |████████████████████████████████| 182 kB 72.6 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 10.8 MB/s 
     |████████████████████████████████| 7.6 MB 70.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████|

In [ ]:
number_classes = 2
model_output = "Jiabo/Roberta_fake_news"

In [ ]:
! wget -nc https://s3.amazonaws.com/assets.datacamp.com/production/course_3629/fake_or_real_news.csv
! rm -rf output

--2022-12-21 11:04:44--  https://s3.amazonaws.com/assets.datacamp.com/production/course_3629/fake_or_real_news.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.43.88, 54.231.236.168, 54.231.164.96, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.43.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30696129 (29M) [text/csv]
Saving to: ‘fake_or_real_news.csv’

fake_or_real_news.c 100%[===================>]  29.27M  7.21MB/s    in 4.1s    

2022-12-21 11:04:49 (7.21 MB/s) - ‘fake_or_real_news.csv’ saved [30696129/30696129]



In [ ]:
import pandas as pd

df = pd.read_csv('/content/fake_or_real_news.csv')

# 02 Data preprocessing

In [ ]:
map_dict = dict()

map_dict['FAKE']=1
map_dict['REAL']=0

In [ ]:
df['label'] = df['label'].map(map_dict).astype('int')

df.head(5)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [ ]:
from sklearn.model_selection import train_test_split

finetune_df, test_df = train_test_split(df, test_size=0.2)


# finetune_df = df.sample(n=1000)
finetune_df = finetune_df[['text', 'label']]

# test_df = df.sample(n=3000)
test_df = test_df[['text', 'label']]

In [ ]:
finetune_df

,text,label
4049,"Carmela Tyrell November 3, 2016 How To Choose ...",1
4653,This is only a preview. Your comment has not y...,1
2380,"SPECIAL TO BUSINESS WEEK, MINDY KATZMAN, AUTH....",1
3543,Here is a review of some of the latest signifi...,1
2203,‹ › Arnaldo Rodgers is a trained and educated ...,1
...,...,...
746,BEIRUT (AP) — Islamic State militants have acc...,0
2650,11/08/2016,1
5011,Millennials ‘Search for Truth’ on Election but...,1
3651,Despite what he told CNN in an interview short...,0


In [ ]:
from datasets import Dataset, DatasetDict

dataset_dict = {'finetune': Dataset.from_pandas(finetune_df),
                'test' : Dataset.from_pandas(test_df)}

dataset = DatasetDict(dataset_dict)

In [ ]:
dataset

DatasetDict({
    finetune: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 5068
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1267
    })
})

# 03 Tokenization

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")



Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

# 04 Finetune BERT

In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                                         num_labels=2)


Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-14-e1ee7db0da93>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=3,
                                  output_dir="output")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=tokenized_datasets["finetune"], 
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5068
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1902
  Number of trainable parameters = 124647170


Step,Training Loss
500,0.083600
1000,0.019900
1500,0.014100


Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1902, training_loss=0.030896465240690767, metrics={'train_runtime': 1535.5445, 'train_samples_per_second': 9.901, 'train_steps_per_second': 1.239, 'total_flos': 4000340485693440.0, 'train_loss': 0.030896465240690767, 'epoch': 3.0})

# 05 Evaluation

In [ ]:
import numpy as np

trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1267
  Batch size = 8


{'eval_loss': 0.02067147195339203,
 'eval_accuracy': 0.9976322020520916,
 'eval_runtime': 40.925,
 'eval_samples_per_second': 30.959,
 'eval_steps_per_second': 3.885,
 'epoch': 3.0}

In [ ]:
!huggingface-cli login
#


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
#https://huggingface.co/docs/transformers/model_sharing
model.push_to_hub(model_output)

Configuration saved in /tmp/tmpxs141gke/config.json
Model weights saved in /tmp/tmpxs141gke/pytorch_model.bin
Uploading the following files to Jiabo/Roberta_fake_news: pytorch_model.bin,config.json


CommitInfo(commit_url='https://huggingface.co/Jiabo/Roberta_fake_news/commit/2926481ddd6186ac4ea90349062fd8002f4474d7', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='2926481ddd6186ac4ea90349062fd8002f4474d7', pr_url=None, pr_revision=None, pr_num=None)